In [ ]:
import os
import urllib.request
import zipfile
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score

# Load the UCI Bike Sharing Dataset
DATA_DIR = "bike_data"
CSV_FILE = os.path.join(DATA_DIR, "day.csv")
ZIP_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip"
ZIP_FILE = "bike_sharing.zip"

print("Loading the UCI Bike Sharing Dataset...")

if os.path.exists(CSV_FILE):
    dframe = pd.read_csv(CSV_FILE, parse_dates=["dteday"])
    print("Dataset loaded successfully from disk.")
else:
    print("Dataset not found locally. Downloading from UCI repository...")

    urllib.request.urlretrieve(ZIP_URL, ZIP_FILE)

    with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)

    dframe = pd.read_csv(CSV_FILE, parse_dates=["dteday"])
    print("Dataset downloaded and loaded successfully.")


Loading the UCI Bike Sharing Dataset...
Dataset not found locally. Downloading from UCI repository...
Dataset downloaded and loaded successfully.


In [ ]:
dframe.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


### Fearture engineering

In [ ]:
# Convert date to datetime
dframe['dteday'] = pd.to_datetime(dframe['dteday'])
dframe['year'] = dframe['dteday'].dt.year
dframe['month'] = dframe['dteday'].dt.month
dframe['day_of_week'] = dframe['dteday'].dt.dayofweek


In [ ]:
# Create features for analysis
# Target variable
target = 'cnt'
# Numerical features (excluding target and instant)
num_features = ['temp', 'atemp', 'hum', 'windspeed', 'year', 'month', 'day_of_week']
# Categorical features
cat_features = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']

In [ ]:
dframe.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,year,month,day_of_week
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985,2011,1,5
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801,2011,1,6
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349,2011,1,0
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562,2011,1,1
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600,2011,1,2


In [ ]:
# Data Preprocessing
processed_df = dframe.copy()

#One-hot encode categorical features
processed_df = pd.get_dummies(processed_df, columns=cat_features, drop_first=True,dtype=int)
#Drop unnecessary columns
processed_df = processed_df.drop(['instant', 'dteday', 'casual', 'registered'], axis=1)

In [ ]:
print(f"Processed dataframe shape: {processed_df.shape}")
print("\nProcessed features:")
print(processed_df.columns.tolist())

Processed dataframe shape: (731, 33)

Processed features:
['temp', 'atemp', 'hum', 'windspeed', 'cnt', 'year', 'month', 'day_of_week', 'season_2', 'season_3', 'season_4', 'yr_1', 'mnth_2', 'mnth_3', 'mnth_4', 'mnth_5', 'mnth_6', 'mnth_7', 'mnth_8', 'mnth_9', 'mnth_10', 'mnth_11', 'mnth_12', 'holiday_1', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'workingday_1', 'weathersit_2', 'weathersit_3']


In [ ]:
processed_df

,temp,atemp,hum,windspeed,cnt,year,month,day_of_week,season_2,season_3,...,holiday_1,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,workingday_1,weathersit_2,weathersit_3
0,0.344167,0.363625,0.805833,0.160446,985,2011,1,5,0,0,...,0,0,0,0,0,0,1,0,1,0
1,0.363478,0.353739,0.696087,0.248539,801,2011,1,6,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.196364,0.189405,0.437273,0.248309,1349,2011,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0.200000,0.212122,0.590435,0.160296,1562,2011,1,1,0,0,...,0,0,1,0,0,0,0,1,0,0
4,0.226957,0.229270,0.436957,0.186900,1600,2011,1,2,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,0.254167,0.226642,0.652917,0.350133,2114,2012,12,3,0,0,...,0,0,0,0,1,0,0,1,1,0
727,0.253333,0.255046,0.590000,0.155471,3095,2012,12,4,0,0,...,0,0,0,0,0,1,0,1,1,0
728,0.253333,0.242400,0.752917,0.124383,1341,2012,12,5,0,0,...,0,0,0,0,0,0,1,0,1,0
729,0.255833,0.231700,0.483333,0.350754,1796,2012,12,6,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Split into features and target
X = processed_df.drop(target, axis=1)
y = processed_df[target]

In [ ]:
# Train-test split
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Model Training and Evaluation
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Function to evaluate model performance
def evaluate_model(model, X_train, X_test, y_train, y_test):
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Cross-validation score
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    cv_r2 = cv_scores.mean()

    return {
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_mae': test_mae,
        'cv_r2': cv_r2,
        'model': model,
        'y_pred': y_test_pred
    }


In [ ]:
# Initialize models
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}


In [ ]:
# Train and evaluate each model
results = {}
for name, model in models.items():
    results[name] = evaluate_model(model, X_train_scaled, X_test_scaled, y_train, y_test)
    print(f"{name} Results:")
    print(f"  Training RMSE: {results[name]['train_rmse']:.2f}")
    print(f"  Test RMSE: {results[name]['test_rmse']:.2f}")
    print(f"  Training R²: {results[name]['train_r2']:.4f}")
    print(f"  Test R²: {results[name]['test_r2']:.4f}")
    print(f"  Test MAE: {results[name]['test_mae']:.2f}")
    print(f"  Cross-Validation R²: {results[name]['cv_r2']:.4f}")

Random Forest Results:
  Training RMSE: 261.62
  Test RMSE: 693.28
  Training R²: 0.9813
  Test R²: 0.8801
  Test MAE: 438.67
  Cross-Validation R²: 0.8469
Gradient Boosting Results:
  Training RMSE: 406.65
  Test RMSE: 680.62
  Training R²: 0.9549
  Test R²: 0.8845
  Test MAE: 473.87
  Cross-Validation R²: 0.8640


In [ ]:
# Compare model performances
performance_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Test RMSE': [results[model]['test_rmse'] for model in results],
    'Test R²': [results[model]['test_r2'] for model in results],
    'Test MAE': [results[model]['test_mae'] for model in results],
    'CV R²': [results[model]['cv_r2'] for model in results]
})

print("\nModel Performance Comparison:")
print(performance_df.sort_values('Test R²', ascending=False))


Model Performance Comparison:
               Model   Test RMSE   Test R²    Test MAE     CV R²
1  Gradient Boosting  680.622778  0.884474  473.874431  0.864011
0      Random Forest  693.281429  0.880136  438.671361  0.846892


In [ ]:
performance_df

,Model,Test RMSE,Test R²,Test MAE,CV R²
0,Random Forest,693.281429,0.880136,438.671361,0.846892
1,Gradient Boosting,680.622778,0.884474,473.874431,0.864011


In [ ]:
# Select the best model based on Test R²
best_model_name = performance_df.sort_values('Test R²', ascending=False).iloc[0]['Model']
best_model = results[best_model_name]['model']
best_predictions = results[best_model_name]['y_pred']


In [ ]:
print(f"\nBest model: {best_model_name}")
print(f"Best model Test R²: {results[best_model_name]['test_r2']:.4f}")


Best model: Gradient Boosting
Best model Test R²: 0.8845
